In [ ]:
import os
import sys
import random
import warnings
import random as rn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import cv2

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from skimage.color import rgb2gray

from keras.models import Model, load_model
from keras.layers import Input,Lambda,Conv2D, Conv2DTranspose,MaxPooling2D,concatenate,Dropout,BatchNormalization,UpSampling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras import backend as K
from keras.losses import binary_crossentropy
from imgaug import augmenters as iaa


from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

## Setting up the seeds to make sure no random things happen

In [ ]:
def init_seeds(seed):
    os.environ['PYTHONHASHSEED'] = '0'

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(seed)

    # Force TensorFlow to use single thread.
    # Multiple threads are a potential source of
    # non-reproducible results.
    # For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)


    # The below tf.set_random_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    return sess




In [ ]:
# Set some parameters
n_pixels = 128
channels = 1
path_train = '../input/train/'
path_test = '../input/test/'

In [ ]:
train=pd.read_csv("../input/train.csv")
print(train.shape)
train.head()

In [ ]:
sample=pd.read_csv("../input/sample_submission.csv")
print(sample.shape)
sample.head()

In [ ]:
depth=pd.read_csv("../input/depths.csv")
print(depth.shape)
depth.head()

In [ ]:
train_ids = train.id.values
test_ids = sample.id.values

In [ ]:
# Get and resize train images and masks
X = np.zeros((len(train_ids), n_pixels, n_pixels, channels), dtype=np.uint8)
y = np.zeros((len(train_ids), n_pixels, n_pixels, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = path_train
    x = np.array(load_img(path + '/images/' + id_+".png",grayscale=False))
#     x = img_to_array(img)[:,:,1]
    x=x[:,:,1]
    x = resize(x, (n_pixels,n_pixels, channels), mode='constant', preserve_range=True)
    X[n] = x
    mask = np.array(load_img(path + '/masks/' + id_+".png",grayscale=True))
    y[n] = resize(mask, (n_pixels,n_pixels,1), mode='constant', preserve_range=True)

print('Done!')

## VALIDATION SPLIT AND STRATIFICATION

Calculating the salt coverage class

In [ ]:
coverage=np.zeros((train_ids.shape[0],))
for i,name in tqdm_notebook(enumerate(train_ids),total=train_ids.shape[0]):
    coverage[i]=np.sum(y[i,:,:,0])

train['coverage']=pd.Series(coverage)/y.shape[1]**2

print(train.shape)
train.head()

In [ ]:
def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train["coverage_class"] = train.coverage.apply(cov_to_class)

print(train.shape)
train.head()

In [ ]:
plt.scatter(train.coverage,train.coverage_class)

In [ ]:
## coverage class numpy array
coverage=train.coverage_class.values
print(coverage.shape)

In [ ]:
def crop_rescale_image(image,mask,state,limit):
    np.random.seed(state)
    h,w=image.shape[0],image.shape[1]
    dy=int(h*limit)
    y0=np.random.randint(0,dy)
    y1=h-y0
    
    np.random.seed(state+5000)
    dx=int(w*limit)
    x0=np.random.randint(0,dx)
    x1=w-x0
    
    cropped_image=resize(image[y0:y1,x0:x1,:],(h,w),mode='constant',preserve_range=True)
    cropped_mask=resize(mask[y0:y1,x0:x1,:],(h,w),mode='constant',preserve_range=True)
    
    return cropped_image,cropped_mask
    
def random_crop_rescale(X,y,limit):
    m=X.shape[0]
    processed_image=np.zeros(X.shape)
    processed_mask=np.zeros(y.shape)
    for i in range(m):
        processed_image[i,:,:,:],processed_mask[i,:,:,:]=crop_rescale_image(X[i,:,:,:],y[i,:,:,:],i,limit)
    
    return processed_image,processed_mask

In [ ]:
# augmentation
print("Before augmentation",X.shape,y.shape,coverage.shape)
# flipping in left/right 

X=np.append(X,[np.fliplr(x) for x in X],axis=0)
y=np.append(y,[np.fliplr(i) for i in y],axis=0)
coverage=np.append(coverage,coverage,axis=0)

print("After Augmentation",X.shape,y.shape,coverage.shape)


np.random.seed(42)
index=np.arange(0,4000)
# shuffling the array
np.random.shuffle(index)

# add gaussian blur to ranomly selected 800 images (20 %)
seq = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])
X=np.append(X,seq.augment_images(X[index[0:1600]]),axis=0)
y=np.append(y,y[0:1600],axis=0)
coverage=np.append(coverage,coverage[0:1600],axis=0)

# doing the random crop and rescale the image.

processed_X,processed_y=random_crop_rescale(X[index[1600:4000]],y[index[1600:4000]],limit=0.25)
X=np.append(X,processed_X,axis=0)
y=np.append(y,processed_y,axis=0)
coverage=np.append(coverage,coverage[index[1600:4000]],axis=0)

print("After Augmentation",X.shape,y.shape,coverage.shape)

In [ ]:
# split

X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.1,stratify=coverage,random_state=42)


print(X_train.shape,X_val.shape)
print(y_train.shape,y_val.shape)

# Train Model
Metrics copied from public kernels

In [ ]:
## metric,copied from https://www.kaggle.com/jcesquiveld/tgs-simple-u-net-baseline

def castF(x):
    return K.cast(x, K.floatx())

def castB(x):
    return K.cast(x, bool)

def iou_loss_core(true,pred):
    intersection = true * pred
    notTrue = 1 - true
    union = true + (notTrue * pred)

    return (K.sum(intersection, axis=-1) + K.epsilon()) / (K.sum(union, axis=-1) + K.epsilon())

def competition_metric(true, pred): #any shape can go

    tresholds = [0.5 + (i*.05)  for i in range(10)]

    #flattened images (batch, pixels)
    true = K.batch_flatten(true)
    pred = K.batch_flatten(pred)
    pred = castF(K.greater(pred, 0.5))

    #total white pixels - (batch,)
    trueSum = K.sum(true, axis=-1)
    predSum = K.sum(pred, axis=-1)

    #has mask or not per image - (batch,)
    true1 = castF(K.greater(trueSum, 1))    
    pred1 = castF(K.greater(predSum, 1))

    #to get images that have mask in both true and pred
    truePositiveMask = castB(true1 * pred1)

    #separating only the possible true positives to check iou
    testTrue = tf.boolean_mask(true, truePositiveMask)
    testPred = tf.boolean_mask(pred, truePositiveMask)

    #getting iou and threshold comparisons
    iou = iou_loss_core(testTrue,testPred) 
    truePositives = [castF(K.greater(iou, tres)) for tres in tresholds]

    #mean of thressholds for true positives and total sum
    truePositives = K.mean(K.stack(truePositives, axis=-1), axis=-1)
    truePositives = K.sum(truePositives)

    #to get images that don't have mask in both true and pred
    trueNegatives = (1-true1) * (1 - pred1) # = 1 -true1 - pred1 + true1*pred1
    trueNegatives = K.sum(trueNegatives) 

    return (truePositives + trueNegatives) / castF(K.shape(true)[0])

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
# loss
# here i will add dice and bce loss
# taken from https://www.kaggle.com/kmader/u-net-with-dice-and-augmentation

def dice_loss(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    score=K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
    return 1.0-score

def dice_and_bce(in_gt, in_pred):
    """combine DICE and BCE"""
    return binary_crossentropy(in_gt, in_pred) + dice_loss(in_gt, in_pred)

## ARCHITECUTE

In [ ]:
inputs = Input((n_pixels,n_pixels,channels))

normalize=True
batch_normalize=True

if normalize:
    s=Lambda(lambda x: x / 255) (inputs)
else:
    s = inputs




# 128 to 64
c1 = Conv2D(16, (3, 3), activation='relu', padding='same') (s)
c1=BatchNormalization()(c1) if batch_normalize else c1
c1 = Conv2D(16, (3, 3), activation='relu', padding='same') (c1)
c1=BatchNormalization()(c1) if batch_normalize else c1
p1 = MaxPooling2D((2, 2)) (c1)

# 64 to 32
c2 = Conv2D(32, (3, 3), activation='relu', padding='same') (p1)
c2=BatchNormalization()(c2) if batch_normalize else c2
c2 = Conv2D(32, (3, 3), activation='relu', padding='same') (c2)
c2=BatchNormalization()(c2) if batch_normalize else c2
p2 = MaxPooling2D((2, 2)) (c2)

# 32 to 16
c3 = Conv2D(64, (3, 3), activation='relu', padding='same') (p2)
c3=BatchNormalization()(c3) if batch_normalize else c3
c3 = Conv2D(64, (3, 3), activation='relu', padding='same') (c3)
c3=BatchNormalization()(c3) if batch_normalize else c3
p3 = MaxPooling2D((2, 2)) (c3)

# 16 to 8
c4 = Conv2D(128, (3, 3), activation='relu', padding='same') (p3)
c4=BatchNormalization()(c4) if batch_normalize else c4
c4 = Conv2D(128, (3, 3), activation='relu', padding='same') (c4)
c4=BatchNormalization()(c4) if batch_normalize else c4
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

# middle layer
c5 = Conv2D(256, (3, 3), activation='relu', padding='same') (p4)
c5=BatchNormalization()(c5) if batch_normalize else c5
c5 = Conv2D(256, (3, 3), activation='relu', padding='same') (c5)
c5=BatchNormalization()(c5) if batch_normalize else c5

# 8 to 16
u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='relu', padding='same') (u6)
c6=BatchNormalization()(c6) if batch_normalize else c6
c6 = Conv2D(128, (3, 3), activation='relu', padding='same') (c6)
c6=BatchNormalization()(c6) if batch_normalize else c6

# 16 to 32
u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='relu', padding='same') (u7)
c7=BatchNormalization()(c7) if batch_normalize else c7
c7 = Conv2D(64, (3, 3), activation='relu', padding='same') (c7)
c7=BatchNormalization()(c7) if batch_normalize else c7

# 32 to 64
u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='relu', padding='same') (u8)
c8=BatchNormalization()(c8) if batch_normalize else c8
c8 = Conv2D(32, (3, 3), activation='relu', padding='same') (c8)
c8=BatchNormalization()(c8) if batch_normalize else c8

# 64 to 128
u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='relu', padding='same') (u9)
c9=BatchNormalization()(c9) if batch_normalize else c9
c9 = Conv2D(16, (3, 3), activation='relu', padding='same') (c9)
c9=BatchNormalization()(c9) if batch_normalize else c9

# 1x1 convolution
outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss=dice_and_bce, metrics=[mean_iou,competition_metric])
model.summary()


In [ ]:
batch=32
ep=30

## copied from https://github.com/jfpuget/LibFM_in_Keras/blob/master/keras_blog.ipynb
# if we do this we cant use mean_iou as the metric due to initializations errors
# try:
#     del sess
# except:
#     pass
# sess = init_seeds(0)


name_model='stratification_unet.h5'

earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint(name_model, verbose=1, save_best_only=True)
reduce_lr=ReduceLROnPlateau(patience=3, min_lr=0.00001, verbose=1)

results = model.fit(X_train, y_train, validation_data=[X_val,y_val], batch_size=batch, epochs=ep, 
                    callbacks=[earlystopper, checkpointer,reduce_lr])

# Test Data
First we'll get the test data. This takes a while, it's 18000 samples.

In [ ]:
# Get and resize test images
X_test = np.zeros((len(test_ids),n_pixels,n_pixels,channels), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(test_ids)):
    path = path_test
    img = load_img(path + '/images/' + id_+".png")
    x = img_to_array(img)[:,:,1]
    sizes_test.append([x.shape[0], x.shape[1]])
    x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)
    X_test[n] = x

print('Done!')

In [ ]:
model = load_model(name_model, custom_objects={'competition_metric': competition_metric,'mean_iou':mean_iou,
                                               'dice_and_bce':dice_and_bce})
print(model.metrics_names)
print(model.evaluate(X_train,y_train))
print(model.evaluate(X_val,y_val))

In [ ]:
# Predict on train, val and test
preds_train = model.predict(X_train, verbose=1)
preds_val = model.predict(X_val, verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [ ]:
# Create list of upsampled test masks
preds_test_upsampled = []
for i in tnrange(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

# Prepare Submission
We need to prepare the submission. 

In [ ]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

pred_dict = {fn:RLenc(np.round(preds_test_upsampled[i])) for i,fn in tqdm_notebook(enumerate(test_ids))}

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')

In [ ]:
sub.head()

In [ ]:
sub.loc[sub['rle_mask']==""].shape